# PFRA 

### _Development Notebook_
--- 

### QA/QC
#### Global Errors Check

This notebook is intended to identify global issues in a model simulation that require follow-up responses from the modeler before the model can be used in *STARRII* PFRA studies.

Global issues currently implemented include:
 
 __1. Check Model inputs__
 
    A. Plan Information
    B. Plan Parameters
    C. Boundary Conditions 
     
__2. Check Results for each domain (2D Flow Area) in the model__
 
    A. Computational Summary & Depth Plots
    B. Domain Extents
    C. High Velocities at cell faces
 
 
Global issues *in development* include:
 - Mapping areas where Courant number exceeds indicated threshold.
 - Mapping areas where iteration counts exceed indicated threshold.

In [1]:
import sys;sys.path.append('../')

In [2]:
from hecrasio.core import ResultsZip
from hecrasio.qaqc import *
import scrapbook as sb

#### Read model output data 

*from AWS*

In [3]:
model_s3path = "s3://azavea/jwx_test/data/zip/DC_F01_NBR_E0004_out.zip"
model = ResultsZip(model_s3path, require_prj=False)

In [4]:
hdfResults_paths = [f for f in model.contents if '.hdf' in f]
assert len([f for f in model.contents if '.hdf' in f]) == 1, "Check files...too many hdf's found"

In [5]:
hdfResults_path = hdfResults_paths[0]
hdfResults_path

'DC_F01_NBR_E0004.p01.hdf'

In [6]:
rasPlan = HDFResultsFile(model, hdfResults_path)

Single domain found...


### 1A. Plan Information

In [7]:
sb.glue(rasPlan.Plan_Information.to_json(), "json")
rasPlan.Plan_Information

,Results
Base Output Interval,15MIN
Computation Time Courant Method,Representative Length/Velocity
Computation Time Step Base,30SEC
Computation Time Step Count To Double,2
Computation Time Step Max Courant,2
Computation Time Step Max Doubling,2
Computation Time Step Max Halving,1
Computation Time Step Min Courant,0.5
Flow Filename,DC_F01_NBR.u01
Flow Title,E0004


### 1B. Plan Parameters

In [8]:
sb.glue(rasPlan.Plan_Parameters.to_json(), "json")
rasPlan.Plan_Parameters

,Results
1D Cores,0
1D Methodology,Finite Difference
1D2D Flow Tolerance,0.1
1D2D MaxIter,0
1D2D Minimum Flow Tolerance,1
1D2D WS Tolerance,0.01
2D Boundary Condition Ramp Up Fraction,[0.1]
2D Boundary Condition Volume Check,[b'False']
2D Cores,0
2D Coriolis,False


### 1C. Boundary Conditions 

#### Input hydrographs

In [9]:
# # Logic to handle one and many domains
# if len(rasPlan.domains) > 1:
#     results = [DomainResults(model, rasPlan, domainID) for domainID in rasPlan.domains]
#     for result in results:
#         plotBCs(result) 
    
# else:
#     results = DomainResults(model, rasPlan, rasPlan.domains[0])
#     plotBCs(results)

### 2A. Computational Summary

In [22]:
non_nan = rasPlan.summary.replace(np.nan, "00:00:00")
sb.glue(non_nan.to_json(), "json")
rasPlan.summary

,Results
Computation Time DSS,00:00:00
Computation Time Total,03:11:15
Maximum WSEL Error,0
Run Time Window,15JUL2019 03:58:00 to 15JUL2019 07:09:15
Solution,Unsteady Finished Successfully
Time Solution Went Unstable,NaN
Time Stamp Solution Went Unstable,Not Applicable
Vol Accounting,Volume Accounting in Acre Feet
Vol Accounting 2D Bdry,0
Vol Accounting Cell End,99553.8


### 2A. Depth Plots

In [ ]:
# try:
#     for result in results:
#         plot_descriptive_stats(result.Describe_Depths, result.Perimeter)
# except:
#     plot_descriptive_stats(results.Describe_Depths, results.Perimeter)

###  2B. Domain Extents
*Boundary Condition locations should be the only areas shown

In [ ]:
# try:
#     for result in results:
#         test = plot_extreme_edges(result.Extreme_Edges, result.Perimeter, mini_map = rasPlan.domain_polys)
# except:
#     plot_extreme_edges(results.Extreme_Edges, results.Perimeter)

###  2C. High Velocities at cell faces

In [ ]:
# try:
#     for result in results:
#         velCheckMain(result, plot_tseries=3)
# except:
#     velCheckMain(results, plot_tseries=3)

# END